## <span class="motutor-highlight motutor-id_3c72f8t-id_af4j17w"><i></i>决策树</span>

决策树是一种通过<span class="motutor-highlight motutor-id_lpvozkb-id_mslkgsp"><i></i>树形结构</span>进行分类的方法。  
在决策树中，树形结构中每个节点表示对分类目标在属性上的一个判断，每个分支代表基于该属性做出的一个判断，最后树形结构中每个叶子节点代表一种分类结果。

<img src="https://imgbed.momodel.cn/decision_tree.png" width=50%>

那其实对于上图中的问题，是否接受这个工作有三个判断条件，哪一个条件更加重要，哪一个没那么重要，其实是由**信息熵**来决定的。**信息熵**用来度量信息量的大小。  
从信息论的角度来看，对信息的度量等于计算信息不确定性的多少。  

In [ ]:
from IPython.display import IFrame
src = 'http://files.momodel.cn/%E5%86%B3%E7%AD%96%E6%A0%9123.pptx'
IFrame('https://view.officeapps.live.com/op/view.aspx?src='+src, width=800, height=600)

假设有 $K$ 个信息，其组成了集合样本 $D$ ，记第 $k$ 个信息发生的概率为 $p_k(1≤k≤K)$。  
这 $K$ 个信息的<span class="motutor-highlight motutor-id_syo1wf2-id_8ng9cas"><i></i>信息熵</span>：  
$$E(D)=-\sum_{k=1}^{K}p_k log_{2} p_k$$

$E(D)$ 的值越小，表示$ D $ 包含的信息越确定，也称$ D $的纯度越高。

需要指出：**所有 $p_k$ 累加起来的和为1**。

当只有两个信息时，熵随概率变化的曲线如图所示

<center><img src="http://imgbed.momodel.cn//20200324130026.png" width=300></center>

我们来动手实现一下信息熵的计算。
$$E(D)=-\sum_{k=1}^{K}p_k log_{2} p_k$$

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
from math import log
import warnings
warnings.filterwarnings("ignore")

In [3]:
def calc_entropy(total_num, count_dict):
    """
    计算信息熵
    :param total_num: 总样本数, 例如总的样本数是 14
    :param count_dict: 每类样本及其对应数目的字典，例如：{'前往游乐场': 9, '不前往游乐场': 5}
    :return: 信息熵
    """
    
    #初始化 ent 为 0
    ent = 0
    # 对于每一个类别
    for n in count_dict.values():
        # 如果属于该类别的样本数大于 0
        if n > 0:
            # 计算概率
            p = n / total_num
            # 计算信息熵
            ent += - p * log(p, 2)
    # 返回信息熵，精确到小数点后 3 位
    return round(ent, 3)


接下来我们读取一个数据集，数据集中是记录了过去14天的天气状况以及是否前往游乐场。

In [4]:
df = pd.read_csv('datasets.csv')
df

,天气,温度,湿度,是否有风,是否前往游乐场
0,晴,>26,>75,否,0
1,晴,<=26,>75,是,0
2,多云,>26,>75,否,1
3,雨,<=26,>75,否,1
4,雨,<=26,>75,否,1
5,雨,<=26,<=75,是,0
6,多云,<=26,<=75,是,1
7,晴,<=26,>75,否,0
8,晴,<=26,<=75,否,1
9,雨,<=26,>75,否,1


<span class="motutor-highlight motutor-id_w3cr191-id_nee6s7v"><i></i>数据标签情况</span>：数据中 14 个样本分为 “游客来游乐场（ 9 个样本）” 和 “游客不来游乐场（ 5 个样本）” 两个类别，即 K = 2。

记 “游客来游乐场” 和 “游客不来游乐场” 的概率分别为 $p_1$ 和 $p_2$ ，显然 $p_1=\frac{9}{14}$，$p_1=\frac{5}{14}$ ，则这 14 个样本所蕴含的信息熵：

$$E(D)=-\sum_{k=1}^{2}p_{k}log_{2}{p_k}=-(\frac{9}{14}×log_{2}{\frac{9}{14}}+\frac{5}{14}×log_{2}{\frac{5}{14}})=0.940$$

In [5]:
# 样本总数
total_num = 14
# 每类样本的数量
count_dict = {'去游乐场': 9, '不去游乐场': 5}

calc_entropy(total_num, count_dict)

0.94

上面我们是一行行数出来的，如何使用代码得到 total_num 和 count_dict ？

In [6]:
df

,天气,温度,湿度,是否有风,是否前往游乐场
0,晴,>26,>75,否,0
1,晴,<=26,>75,是,0
2,多云,>26,>75,否,1
3,雨,<=26,>75,否,1
4,雨,<=26,>75,否,1
5,雨,<=26,<=75,是,0
6,多云,<=26,<=75,是,1
7,晴,<=26,>75,否,0
8,晴,<=26,<=75,否,1
9,雨,<=26,>75,否,1


我们可以用下面这种方式对 dataframe 的数据按条件进行筛选。

In [7]:
# 例如：按是否前往游乐场==0 进行筛选
df[df['是否前往游乐场']== 0 ]


,天气,温度,湿度,是否有风,是否前往游乐场
0,晴,>26,>75,否,0
1,晴,<=26,>75,是,0
5,雨,<=26,<=75,是,0
7,晴,<=26,>75,否,0
13,雨,<=26,>75,是,0


我们可以用下面这种方式对查看数据的行数和列数。

In [8]:
df.shape

(14, 5)

使用上面的方法，可以得到计算信息熵所需的总样本数，以及每类样本及其对应数目的字典，然后计算信息熵。

In [9]:
# 总样本数
total_num = df.shape[0]

# 前往游乐场的样本数目
num_go_to_play = df[df['是否前往游乐场']== 1 ].shape[0]

# 不前往游乐场的样本数目
num_not_go_to_play = df[df['是否前往游乐场']== 0 ].shape[0]

# 每类样本及其对应数目的字典
count_dict = {'前往': num_go_to_play,
              '不前往': num_not_go_to_play}

# 计算信息熵
entropy = calc_entropy(total_num, count_dict)
entropy

0.94

<b>计算天气状况所对应的信息熵</b>：  
<span class="motutor-highlight motutor-id_av3d6ny-id_ukw8ivl"><i></i>天气状况的三个属性</span>记为 $a_0=“晴”$ ，$a_1=“多云”$ ，$a_2=“雨”$ ，  
属性取值为 $a_i$ 对应分支节点所包含子样本集记为 $D_i$ ，该子样本集包含样本数量记为 $|D_i|$ 。

|天气属性取值$a_i$|“晴”|“多云”|“雨”|
|:--:|:--:|:--:|:--:|
|对应样本数$|D_i|$|5|4|5|
|正负样本数量|（2+，3-）|（4+，0-）|（3+，2-）|

计算天气状况每个属性值的信息熵：

$“晴”：E(D_0)=-(\frac{2}{5}×log_{2}{\frac{2}{5}}+\frac{3}{5}×log_{2}{\frac{3}{5}})=0.971$

$“多云”：E(D_1)=-(\frac{4}{4}×log_{2}{\frac{4}{4}})=0$

$“雨”：E(D_2)=-(\frac{3}{5}×log_{2}{\frac{3}{5}}+\frac{2}{5}×log_{2}{\frac{2}{5}})=0.971$

现在，我们来编写代码进行完成上面的计算。

首先，我们可以使用下面的写法，对 Dataframe 进行多个条件的筛选。

In [10]:
# 筛选出 天气为晴并且去游乐场的样本数据
df[(df['天气']=='晴') & (df['是否前往游乐场']== 1 )]


,天气,温度,湿度,是否有风,是否前往游乐场
8,晴,<=26,<=75,否,1
10,晴,<=26,<=75,是,1


然后，我们是使用上面的筛选方法，分别计算不同天气下的信息熵。

In [11]:
# 天气为晴的总天数
total_num_sun = df[df['天气']=='晴'].shape[0]

# 前往游乐场的样本数目
num_go_to_play = df[(df['天气']=='晴') & (df['是否前往游乐场']== 1 )].shape[0]

# 不前往游乐场的样本数目
num_not_go_to_play = df[(df['天气']=='晴') & (df['是否前往游乐场']== 0 )].shape[0]


# 天气为晴时，去游乐场和不去游乐场的人数
count_dict_sun = {'前往': num_go_to_play ,
                  '不前往': num_not_go_to_play}
print(count_dict_sun)

# 计算天气-晴 的信息熵
ent_sun = calc_entropy(total_num_sun, count_dict_sun)
print('天气-晴 的信息熵为：%s' % ent_sun)


{'前往': 2, '不前往': 3}
天气-晴 的信息熵为：0.971


In [12]:
# 天气为多云的总天数
total_num_cloud = df[df['天气']=='多云'].shape[0]

# 天气为多云时，去游乐场和不去游乐场的人数
count_dict_cloud = {'前往':df[(df['天气']=='多云') & (df['是否前往游乐场']== 1 )].shape[0],
                    '不前往':df[(df['天气']=='多云') & (df['是否前往游乐场']== 0 )].shape[0]}
print(count_dict_cloud)

# 计算天气-多云 的信息熵
ent_cloud = calc_entropy(total_num_cloud, count_dict_cloud)
print('天气-多云 的信息熵为：%s' % ent_cloud)


{'前往': 4, '不前往': 0}
天气-多云 的信息熵为：0.0


In [13]:
# 天气为雨的总天数
total_num_rain = df[df['天气']=='雨'].shape[0]

# 天气为雨时，去游乐场和不去游乐场的人数
count_dict_rain = {'前往':df[(df['天气']=='雨') & (df['是否前往游乐场']== 1 )].shape[0],
              '不前往':df[(df['天气']=='雨') & (df['是否前往游乐场']== 0 )].shape[0]}
print(count_dict_rain)

# 计算天气-雨 的信息熵
ent_rain = calc_entropy(total_num_rain, count_dict_rain)
print('天气-雨 的信息熵为：%s' % ent_rain)


{'前往': 3, '不前往': 2}
天气-雨 的信息熵为：0.971
